In [2]:
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt
import numba as nb
import sys

## plotting

    plt.figure(figsize=(17,10))
    plt.plot(time, y_sol, label ='$P_{x_{\epsilon = 8}}$', linestyle = ':', color = 'purple')
    #plt.title("cheese - the biggest")
    plt.xlabel(r"t ($MeV^{-1}$)", fontsize=20)
    plt.ylabel("y")
    plt.legend(loc = "upper right", prop={'size': 22})
    # Save the figure with a relative path
    relative_path = "custom_directory"  # Customize the directory and filename
    directory = os.path.dirname(relative_path)     # Create the directory if it doesn't exist
    if not os.path.exists(directory):
        os.makedirs(directory)
    plt.savefig(f'{relative_path}/thecheesethatcould.png)
    plt.show()

## function

    def F(t,y):
        return -t*y

## FFT

    t = np.linspace(0,20,2**15)
    dt = t[1]-t[0]
    frequency = np.fft.rfftfreq(len(t), dt)
    y = np.sin(t)
    FT = np.fft.rfft(y)

    avs = np.abs(FT)**2
    plt.semilogx(frequency, avs)

## loops

    while k < N:
        if k < 10:
            s = s + k
        else:
            s = k**2

    for k in range(25):
        s = 2 * k**3
        print(s)

## Math

    np.mean(b)
    np.std(b)
    np.sqrt(len(b))
    np.argmin(f(x))

## random

    rg = np.random.default_rng()
    random_numbers = rg.standard_normal(10000)
    a = rg.random(10)

## Runge-Kutta
### if you have a model:
    def model(t):
        y = np.sqrt(t**2 + 25)
        return y

    def F(t,y):
        F = t/y
        return F
    
### if not, proceed here
reference WS12 for more info

$\frac{dx}{dt}=\frac{p}{m}$

$\frac{dp}{dt} = -kx^3$

    def derivs(t, y): 
        #y[0] = x
        #y[1] = p
        #think of y as our dependent variables and t as independent
        derivs = np.zeros(2)
        derivs[0] = y[1]/m #dx/dt
        derivs[1] = -k*y[0]**3 #dp/dt

        return derivs

    def fourthorderRK(t, y, dt, F):
        k1 = dt * F(t, y)
        y_est = y + 0.5*k1
        k2 = dt * F(t+dt/2, y_est)
        y_est2 = y + 0.5*k2
        k3 = dt * F(t+dt/2, y_est2)
        y_est3 = y + k3
        k4 = dt * F(t+dt, y_est3)

        step = (1/6)*(k1 + 2*k2 + 2*k3 + k4)

        return step

    lotsofts = np.linspace(0,2,10000)
    actualy = model(lotsofts)

    def fourthorderstepper(N):
        t = np.linspace(0,2,N)
        y = np.zeros(N)
        y[0] = 5
        dt = t[1] - t[0]

        for i in range(1,N):
            y[i] = y[i-1] + fourthorderRK(t[i-1], y[i-1], dt, F)

        return t, y, dt
        

### MIT RK Variant

    def dynamic(y,t):
        eps = 0.2
        # Modify this function to return the dynamics of the system
        # y is a list of the states
        # t is the time
        # dydt is the derivative of the states
        dydt = np.array([y[1], -y[0]**3-eps*y[1]]) #encode dynamics xdot = f(x) here
        return dydt

    def TC_Simulate(Mode,initialCondition,time_bound):
        # This function simulates the system for a given initial condition and time bound
        time_step = 0.05
        time_bound = float(time_bound)

        number_points = int(np.ceil(time_bound/time_step))
        t = [i*time_step for i in range(0,number_points)]
        if t[-1] != time_step:
            t.append(time_bound)
        newt = []

        for step in t:
            newt.append(float(format(step, '.2f')))
        t = newt

        sol = odeint(dynamic, initialCondition, t, hmax=time_step)

        # Construct the final output
        trace = []
        for j in range(len(t)):
            #print t[j], current_psi
            tmp = []
            tmp.append(t[j])
            tmp.append(float(sol[j, 0]))
            tmp.append(float(sol[j, 1]))
            trace.append(tmp)
        return trace

    if __name__ == "__main__":
        figure, axes = plt.subplots()
        x0 = 5.0
        y0 = 0.0
        plt.scatter(x0, y0, marker="x", c='g')
        sol = TC_Simulate("Default", [x0, y0], 200.0)
        time = [row[0] for row in sol]
        a = [row[1] for row in sol]
        b = [row[2] for row in sol]
        plt.title("Trajectory Evolution for x = [3,0]")
        plt.xlabel("$x_1$")
        plt.ylabel("$x_2$")
        plt.plot(a, b, "-g")
        plt.savefig("traj_evolve.png")
        # countor plot
        # figure, axes = plt.subplots()
        x = np.arange(-5, 5, 0.1) # q
        y = np.arange(-5, 5, 0.1) # qdot

        x, y = np.meshgrid(x, y)
        V = 0.0 # replace this with the Lyapunov function expression
        cs = plt.contourf(x, y, V, levels=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10])
        cs.cmap.set_over('red')
        cs.cmap.set_under('blue')
        cs.changed()
        plt.savefig("traj_evolve_with_V_contours.png")

## Chi-squared

This code investigates how chi-squared (measure of the goodness of a fit, varies as a function of a parameter in our model. It takes into account sigma, which is an array of error bars on the y data.

    def y_model(t, v_0):
        y = v_0*t
        return y

    def chi_squared(t, y, s, v_0):
        c=0
        for i in range(len(t)):
            c = c + ((y[i] - y_model(t[i],v_0))/s[i])**2
        return c

    v_values = np.linspace(5,6,100)

    chisquaredvalues = np.zeros(100)

    for i in range(len(v_values)):
        chisquaredvalues[i] = chi_squared(t, y, sigma, v_values[i])

    plt.figure()
    plt.plot(v_values, chisquaredvalues)
    plt.xlabel("$v_0$")
    plt.ylabel("$\chi^2$")
    plt.show()

## CPU

%time print(craz(1000000)) #craz is a function

### numba

we introduce ``numba``, which is a Python library that will speed up significantly calculations that primarily use ``numpy``.  Here's what you'll need to do (and don't forget to run the cells after you make the change).
- First ``import numba as nb``.  Add this to the rest of your import statements
- Next, in front of every function (all 4 of them so far), we're going to include the ``numba`` "wrapper" that tells Python to do its magic with the functions to make them run much faster.  For each function, you'll add the line ``@nb.njit()`` to tell it to compile the function and attempt to make it run as fast as more basic code.  For example, the first two lines of the first cell of functions should look like:

``@nb.njit()``

``def F(t,y):``

- After you do this four times, run your code.  You'll notice that there's not much difference the first time we run our timed command.  However, run it again and you'll see significant improvement.  Why?  The first time, Python is compiling the function into something that can run faster, so the second time it does not need to do this step.

## Coordinate Transformations

#### spherical to cartesian (physics)

    def sphereTOcart(r, theta, phi):
        x = 0
        y = 0
        z = 0

        x = r * np.sin(theta) * np.cos(phi)
        y = r*np.sin(theta)*np.sin(phi)
        z = r*np.cos(theta)

        return x, y, z
    
#### cartesian to spherical (physics)

    def cartTOsphere(x, y, z):
        r = 0
        theta = 0
        phi = 0

        r = np.sqrt(x**2 + y**2 + z**2)
        theta = np.arccos(z/(np.sqrt(x**2 + y**2 + z**2)))
        phi = np.arctan(y/x)

        return r, theta, phi
    
#### cartesian to cylindrical    

    def cartTOcyl(x, y, z):
        r = 0
        theta = 0
        z = 0

        r = np.sqrt(x**2 + y**2)
        theta = np.arctan(y/x)
        z = z

        return r, theta, z
    
#### cylindrical to cartesian

    def cartTOcyl(r, theta, z):
        x = 0
        y = 0
        z = 0

        x = r * np.cos(theta)
        y = r * np.sin(theta)
        z = z

        return x, y, z

## Extra

- bootstrap: USD/PHYS371/WS7 Bootstrap
- countour plot: USD/PHYS371/WS8
- ising model: USD/PHYS371/ising.py
- MCMC: USD/PHYS371/WS15